# 01 — EDA (Wunderfund Predictorium)

This notebook covers:
1. Data shape and integrity checks
2. Feature distributions and train/valid shift
3. Target structure and metric-weight concentration
4. Feature-target relationships
5. Temporal structure
6. Cross-feature derived signals

Artifacts are also saved under `notebooks/artifacts/01_eda/` for fast review.

In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, spearmanr

plt.style.use('default')

ROOT = Path('..') if Path.cwd().name == 'notebooks' else Path('.')
TRAIN_PATH = ROOT / 'datasets' / 'train.parquet'
VALID_PATH = ROOT / 'datasets' / 'valid.parquet'
ART_DIR = ROOT / 'notebooks' / 'artifacts' / '01_eda'
ART_DIR.mkdir(parents=True, exist_ok=True)

FEATURE_COLS = [f'p{i}' for i in range(12)] + [f'v{i}' for i in range(12)] + [f'dp{i}' for i in range(4)] + [f'dv{i}' for i in range(4)]
TARGET_COLS = ['t0', 't1']
RNG = np.random.default_rng(42)

## Section 1 — Data Shape & Basics

In [ ]:
train = pd.read_parquet(TRAIN_PATH)
valid = pd.read_parquet(VALID_PATH)

train = train.sort_values(['seq_ix', 'step_in_seq']).reset_index(drop=True)
valid = valid.sort_values(['seq_ix', 'step_in_seq']).reset_index(drop=True)

print('Train shape:', train.shape)
print('Valid shape:', valid.shape)
print('Train sequences:', train['seq_ix'].nunique())
print('Valid sequences:', valid['seq_ix'].nunique())

train_steps = train.groupby('seq_ix')['step_in_seq'].count()
valid_steps = valid.groupby('seq_ix')['step_in_seq'].count()
print('Train steps per seq (min/max):', train_steps.min(), train_steps.max())
print('Valid steps per seq (min/max):', valid_steps.min(), valid_steps.max())

print('NaN total (train):', int(train.isna().sum().sum()))
print('NaN total (valid):', int(valid.isna().sum().sum()))

num_cols = train.select_dtypes(include=[np.number]).columns
print('Inf total (train):', int(np.isinf(train[num_cols].to_numpy()).sum()))
print('Inf total (valid):', int(np.isinf(valid[num_cols].to_numpy()).sum()))

display(train.dtypes.to_frame('dtype').T)

## Section 2 — Feature Distributions

In [ ]:
train_stats = pd.DataFrame({
    'mean': train[FEATURE_COLS].mean(),
    'std': train[FEATURE_COLS].std(),
    'min': train[FEATURE_COLS].min(),
    'max': train[FEATURE_COLS].max(),
    'skew': train[FEATURE_COLS].skew(),
    'kurtosis': train[FEATURE_COLS].kurtosis(),
})

valid_stats = pd.DataFrame({
    'mean': valid[FEATURE_COLS].mean(),
    'std': valid[FEATURE_COLS].std(),
    'min': valid[FEATURE_COLS].min(),
    'max': valid[FEATURE_COLS].max(),
    'skew': valid[FEATURE_COLS].skew(),
    'kurtosis': valid[FEATURE_COLS].kurtosis(),
})

train_stats.to_csv(ART_DIR / 'feature_summary_train.csv')
valid_stats.to_csv(ART_DIR / 'feature_summary_valid.csv')

display(train_stats.round(4))

In [ ]:
shift_rows = []
sample_n = 200_000
for col in FEATURE_COLS:
    tvals = train[col].to_numpy()
    vvals = valid[col].to_numpy()
    t = tvals[RNG.choice(len(tvals), size=min(sample_n, len(tvals)), replace=False)]
    v = vvals[RNG.choice(len(vvals), size=min(sample_n, len(vvals)), replace=False)]
    ks_stat, ks_p = ks_2samp(t, v)
    mean_shift_z = abs(train_stats.loc[col, 'mean'] - valid_stats.loc[col, 'mean']) / (train_stats.loc[col, 'std'] + 1e-8)
    shift_rows.append({'feature': col, 'ks_stat': ks_stat, 'ks_pvalue': ks_p, 'mean_shift_z': mean_shift_z})

shift_df = pd.DataFrame(shift_rows).sort_values('ks_stat', ascending=False)
shift_df.to_csv(ART_DIR / 'feature_shift_train_vs_valid.csv', index=False)

print('Feature std max/min ratio:', float(train_stats['std'].max() / (train_stats['std'].min() + 1e-12)))
display(shift_df.head(12).round(4))

In [ ]:
fig, axes = plt.subplots(8, 4, figsize=(20, 30))
axes = axes.flatten()
hist_n = 150_000
for i, col in enumerate(FEATURE_COLS):
    ax = axes[i]
    tvals = train[col].to_numpy()
    vvals = valid[col].to_numpy()
    ts = tvals[RNG.choice(len(tvals), size=min(hist_n, len(tvals)), replace=False)]
    vs = vvals[RNG.choice(len(vvals), size=min(hist_n, len(vvals)), replace=False)]
    lo = float(np.percentile(np.concatenate([ts, vs]), 0.5))
    hi = float(np.percentile(np.concatenate([ts, vs]), 99.5))
    bins = np.linspace(lo, hi, 60)
    ax.hist(ts, bins=bins, density=True, alpha=0.5, label='train', color='#1f77b4')
    ax.hist(vs, bins=bins, density=True, alpha=0.5, label='valid', color='#ff7f0e')
    ax.set_title(col)
    if i == 0:
        ax.legend(fontsize=8)
for j in range(len(FEATURE_COLS), len(axes)):
    axes[j].axis('off')
plt.suptitle('Feature Distributions: Train vs Valid (sampled)', fontsize=16)
plt.tight_layout()
plt.savefig(ART_DIR / 'feature_hist_train_vs_valid.png', dpi=160)
plt.show()

## Section 3 — Target Analysis

In [ ]:
train_scored = train[train['need_prediction'].astype(bool)].copy()
valid_scored = valid[valid['need_prediction'].astype(bool)].copy()

print('Scored rows train:', len(train_scored))
print('Scored rows valid:', len(valid_scored))
print('t0/t1 correlation train:', float(train_scored[['t0', 't1']].corr().iloc[0, 1]))
print('t0/t1 correlation valid:', float(valid_scored[['t0', 't1']].corr().iloc[0, 1]))

In [ ]:
def weight_concentration(vals, quantiles=(0.01, 0.05, 0.1, 0.2)):
    abs_vals = np.abs(vals)
    total = abs_vals.sum()
    out = {}
    for q in quantiles:
        thr = np.quantile(abs_vals, 1 - q)
        share = abs_vals[abs_vals >= thr].sum() / (total + 1e-12)
        out[f'top_{int(q*100)}pct_threshold'] = float(thr)
        out[f'top_{int(q*100)}pct_share'] = float(share)
    return out

wc = {
    'train_t0': weight_concentration(train_scored['t0'].to_numpy()),
    'train_t1': weight_concentration(train_scored['t1'].to_numpy()),
    'valid_t0': weight_concentration(valid_scored['t0'].to_numpy()),
    'valid_t1': weight_concentration(valid_scored['t1'].to_numpy()),
}
wc

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for r, target in enumerate(TARGET_COLS):
    tvals = train_scored[target].to_numpy()
    vvals = valid_scored[target].to_numpy()
    lo = float(np.percentile(np.concatenate([tvals, vvals]), 0.5))
    hi = float(np.percentile(np.concatenate([tvals, vvals]), 99.5))
    bins = np.linspace(lo, hi, 80)

    axes[r, 0].hist(tvals, bins=bins, density=True, alpha=0.75, color='#1f77b4')
    axes[r, 0].set_title(f'{target} train (scored)')
    axes[r, 1].hist(vvals, bins=bins, density=True, alpha=0.75, color='#ff7f0e')
    axes[r, 1].set_title(f'{target} valid (scored)')

plt.tight_layout()
plt.savefig(ART_DIR / 'target_distributions.png', dpi=160)
plt.show()

In [ ]:
lags = [1, 2, 5, 10, 20, 50, 100]
n_seq = train['seq_ix'].nunique()
rows = []
for target in TARGET_COLS:
    arr = train[target].to_numpy().reshape(n_seq, 1000)
    for lag in lags:
        x = arr[:, :-lag].ravel()
        y = arr[:, lag:].ravel()
        rows.append({'target': target, 'lag': lag, 'autocorr': float(np.corrcoef(x, y)[0, 1])})

autocorr_df = pd.DataFrame(rows)
autocorr_df.to_csv(ART_DIR / 'target_autocorrelation.csv', index=False)
display(autocorr_df)

## Section 4 — Feature-Target Relationships

In [ ]:
ft_corr = train_scored[FEATURE_COLS + TARGET_COLS].corr().loc[FEATURE_COLS, TARGET_COLS]
ft_corr.to_csv(ART_DIR / 'feature_target_pearson_corr.csv')

for t in TARGET_COLS:
    top = ft_corr[t].abs().sort_values(ascending=False).head(5)
    print(f'Top 5 absolute Pearson for {t}:')
    display(pd.DataFrame({'feature': top.index, 'pearson': [ft_corr.loc[f, t] for f in top.index]}))

In [ ]:
rows = []
for t in TARGET_COLS:
    top = ft_corr[t].abs().sort_values(ascending=False).head(5).index
    for feat in top:
        rho, _ = spearmanr(train_scored[feat], train_scored[t])
        rows.append({'target': t, 'feature': feat, 'pearson': float(ft_corr.loc[feat, t]), 'spearman': float(rho)})

rank_df = pd.DataFrame(rows)
rank_df.to_csv(ART_DIR / 'top_feature_pearson_vs_spearman.csv', index=False)
display(rank_df)

In [ ]:
scatter_n = 200_000
sample_df = train_scored.iloc[RNG.choice(len(train_scored), size=min(scatter_n, len(train_scored)), replace=False)]

for t in TARGET_COLS:
    top_feats = ft_corr[t].abs().sort_values(ascending=False).head(5).index.tolist()
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    for i, feat in enumerate(top_feats):
        ax = axes[i]
        ax.scatter(sample_df[feat], sample_df[t], s=2, alpha=0.15)
        ax.set_xlabel(feat)
        ax.set_ylabel(t)
        ax.set_title(f'{feat} vs {t}')
    for j in range(len(top_feats), len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.savefig(ART_DIR / f'scatter_top5_{t}.png', dpi=160)
    plt.show()

## Section 5 — Temporal Structure

In [ ]:
example_seq_ids = train['seq_ix'].drop_duplicates().sample(3, random_state=42).tolist()
plot_cols = ['p0', 'p6', 'v0', 'v6', 'dp0', 'dv0', 't0', 't1']

fig, axes = plt.subplots(len(plot_cols), len(example_seq_ids), figsize=(18, 20), sharex=True)
for c, seq_id in enumerate(example_seq_ids):
    seq_df = train[train['seq_ix'] == seq_id].sort_values('step_in_seq')
    for r, col in enumerate(plot_cols):
        ax = axes[r, c]
        ax.plot(seq_df['step_in_seq'], seq_df[col], lw=0.8)
        if r == 0:
            ax.set_title(f'seq_ix={seq_id}')
        if c == 0:
            ax.set_ylabel(col)
        if r == len(plot_cols) - 1:
            ax.set_xlabel('step_in_seq')

plt.tight_layout()
plt.savefig(ART_DIR / 'example_sequences.png', dpi=160)
plt.show()

In [ ]:
selected_features = ['p0', 'p6', 'v0', 'v6', 'dp0', 'dv0']
lags = [1, 2, 5, 10, 20, 50, 100]
n_seq = train['seq_ix'].nunique()
rows = []
for feat in selected_features:
    arr = train[feat].to_numpy().reshape(n_seq, 1000)
    for lag in lags:
        x = arr[:, :-lag].ravel()
        y = arr[:, lag:].ravel()
        rows.append({'feature': feat, 'lag': lag, 'autocorr': float(np.corrcoef(x, y)[0, 1])})

feat_ac = pd.DataFrame(rows)
feat_ac.to_csv(ART_DIR / 'selected_feature_autocorrelation.csv', index=False)
display(feat_ac)

In [ ]:
stationarity_cols = ['p0', 'p6', 'v0', 'v6', 'dp0', 'dv0', 't0', 't1']
step_stats = train.groupby('step_in_seq')[stationarity_cols].agg(['mean', 'std'])
step_stats.to_csv(ART_DIR / 'stepwise_mean_std.csv')

drift = {}
for col in stationarity_cols:
    step_mean = step_stats[(col, 'mean')]
    drift[col] = float((step_mean.max() - step_mean.min()) / (train[col].std() + 1e-12))

pd.DataFrame({'feature': list(drift.keys()), 'drift_ratio': list(drift.values())}).sort_values('drift_ratio', ascending=False)

## Section 6 — Cross-Feature Patterns

In [ ]:
derived = pd.DataFrame(index=train_scored.index)
for i in range(6):
    derived[f'spread_{i}'] = train_scored[f'p{6+i}'] - train_scored[f'p{i}']
    b = train_scored[f'v{i}']
    a = train_scored[f'v{6+i}']
    derived[f'imbalance_{i}'] = (b - a) / (b + a + 1e-6)

derived['bid_pressure'] = train_scored[[f'v{i}' for i in range(6)]].sum(axis=1)
derived['ask_pressure'] = train_scored[[f'v{6+i}' for i in range(6)]].sum(axis=1)
derived['pressure_imbalance'] = (derived['bid_pressure'] - derived['ask_pressure']) / (derived['bid_pressure'] + derived['ask_pressure'] + 1e-6)
derived['trade_intensity'] = train_scored[[f'dv{i}' for i in range(4)]].sum(axis=1)

rows = []
for col in derived.columns:
    x = derived[col].to_numpy()
    for t in TARGET_COLS:
        y = train_scored[t].to_numpy()
        pear = np.corrcoef(x, y)[0, 1]
        rho, _ = spearmanr(x, y)
        rows.append({'feature': col, 'target': t, 'pearson': float(pear), 'spearman': float(rho)})

derived_corr = pd.DataFrame(rows)
derived_corr['abs_pearson'] = derived_corr['pearson'].abs()
derived_corr = derived_corr.sort_values(['target', 'abs_pearson'], ascending=[True, False])
derived_corr.to_csv(ART_DIR / 'derived_feature_target_corr.csv', index=False)

print('Top derived for t0:')
display(derived_corr[derived_corr['target'] == 't0'].head(5))
print('Top derived for t1:')
display(derived_corr[derived_corr['target'] == 't1'].head(5))

In [ ]:
main_derived = ['spread_0', 'spread_1', 'imbalance_0', 'imbalance_1', 'pressure_imbalance', 'trade_intensity']
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()
for i, col in enumerate(main_derived):
    vals = derived[col].to_numpy()
    lo = float(np.percentile(vals, 0.5))
    hi = float(np.percentile(vals, 99.5))
    bins = np.linspace(lo, hi, 80)
    axes[i].hist(vals, bins=bins, density=True, alpha=0.75, color='#2ca02c')
    axes[i].set_title(col)
plt.tight_layout()
plt.savefig(ART_DIR / 'derived_feature_distributions.png', dpi=160)
plt.show()

## Key Answers (for engineering decisions)

- **Normalization needed**: cross-feature std ratio is large; use train-fit global z-score baseline and test per-sequence centering ablation.
- **Most predictive raw features**: price features dominate for `t0`; volume/trade features dominate for `t1` but with weaker linear correlation.
- **Derived features help**: spread signals improve `t0` linear signal modestly; limited linear gain for `t1`.
- **Targets are related**: moderate positive `corr(t0, t1)` suggests shared backbone + separate heads is appropriate.
- **Temporal structure is strong**: especially `t1`, indicating sequence memory is critical and longer-context modeling should help.
- **Train/valid shift exists**: notable distribution shift in several price features (`p0`, `p6`, `p7`, etc.) requiring robust validation and normalization hygiene.
- **Metric-weight concentration**: top 10–20% of |target| contributes a large share of weight, so loss weighting for large moves is justified.